## 缺失数据
### 一、缺失值的统计和删除
#### 1. 缺失信息的统计

缺失数据可以使用`isna`或`isnull`（两个函数没有区别）来查看每个单元格是否缺失，结合`mean`可以计算出每列缺失值的比例：

In [1]:
%cd drive/MyDrive/
!ls


/content/drive/MyDrive
'Colab Notebooks'   data


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/learn_pandas.csv',
          usecols = ['Grade', 'Name', 'Gender', 'Height',
                         'Weight', 'Transfer'])
df.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [5]:
df.head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N


In [4]:
# 查看缺失的比例
df.isna().mean()

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

如果想要查看某一列缺失或者非缺失的行，可以利用`Series`上的`isna`或者`notna`进行布尔索引。

In [6]:
df[df.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


同时对几个列，检索出全部为缺失或者至少有一个缺失或者没有缺失的行，可以使用`isna,notna`和`any,all`的组合。

In [8]:
# 对身高、体重和转系情况这3列分别进行这三种情况的检索
sub_set = df[['Height', 'Weight', 'Transfer']]

# 全部缺失
df[sub_set.isna().all(1)]  

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [9]:
# 至少有一个缺失
df[sub_set.isna().any(1)].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN
21,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N


In [10]:
# 没有缺失
df[sub_set.notna().all(1)].head()

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


#### 2. 缺失信息的删除

数据处理中经常需要根据缺失值的大小、比例或其他特征来进行样本或列特征的删除，可以使用`dropna`函数来进行操作。

`dropna`的主要参数：
* 轴方向`axis`（默认为0，即删除行）、
* 删除方式`how`、（`how`,`any`）
* 删除的非缺失值个数阈值`thresh`（**非缺失值**没有达到这个数量的相应维度会被删除）
* 备选的删除子集`subset`

In [11]:
# 删除身高体重至少有一个缺失的行

res = df.dropna(how='any', subset=['Height','Weight'])
res.head()


,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


In [12]:
# 删除超过15个缺失值的列

res = df.dropna(1, thresh=df.shape[0]-15)
res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


In [13]:
# 也可以使用布尔索引来完成相关操作
res = df.loc[df[['Height', 'Weight']].notna().all(1)]

res.shape


(174, 6)

In [14]:
res = df.loc[:, ~(df.isna().sum()>15)]

res.head()

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


### 二、缺失值的填充和插值
#### 1. 利用`fillna`进行填充
在`fillna`中有三个参数是常用的：`value,method,limit`。其中，`value`为填充值，可以实标量，也可以是索引到元素的字典映射；`method`为填充方法，有用前面的元素填充`ffill`和用后面的元素填充`bfill`两种类型，`limit`参数表示连续缺失值的最大填充次数。

In [15]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan],
                list('aaabcd'))

In [16]:
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [19]:
# 连续出现的缺失，最多填充一次
s.fillna(method='ffill')

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [18]:
s.fillna(s.mean())

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [20]:
# 通过索引映射填充的值
s.fillna({'a':100, 'd':200})

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

In [21]:
# 可先分组后再进行操作
# 根据年级进行身高的均值填充
df.groupby('Grade')['Height'].transform(
              lambda x: x.fillna(x.mean())).head()

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

#### 2. 插值函数
对于`interpolate`而言，除了插值方法（默认为 `linear` 线性插值）之外，有与 `fillna` 类似的两个常用参数，一个是控制方向的 `limit_direction` ，另一个是控制最大连续缺失值插值个数的 `limit` 。其中，限制插值的方向默认为 `forward` ，这与 `fillna` 的 `method` 中的 `ffill` 是类似的，若想要后向限制插值或者双向限制插值可以指定为 `backward` 或 `both`。 

In [22]:
s = pd.Series([np.nan, np.nan, 1,
        np.nan, np.nan, np.nan,
        2, np.nan, np.nan])

s.values

array([nan, nan,  1., nan, nan, nan,  2., nan, nan])

In [23]:
# 在默认线性插值法下分别进行backward和双向限制插值
# 同时限制最大连续条数为1
res = s.interpolate(limit_direction='backward', limit=1)
res.values

array([ nan, 1.  , 1.  ,  nan,  nan, 1.75, 2.  ,  nan,  nan])

In [24]:
res = s.interpolate(Limit_direction='both', limit=1)
res.values

array([ nan,  nan, 1.  , 1.25,  nan,  nan, 2.  , 2.  ,  nan])

In [25]:
res = s.interpolate(limit_direction='both', limit=1)
res.values

array([ nan, 1.  , 1.  , 1.25,  nan, 1.75, 2.  , 2.  ,  nan])

In [26]:
# 最近邻插补，即缺失值的元素和离他最近的非缺失值元素一样
s.interpolate('nearest').values

array([nan, nan,  1.,  1.,  1.,  2.,  2., nan, nan])

In [27]:
# 索引插值，即根据索引大小进行线性插值
# 构造不等间距的索引
s = pd.Series([0,np.nan,10],index=[0,1,10])
s

0      0.0
1      NaN
10    10.0
dtype: float64

In [28]:
# 默认的线性插值，等价于计算中点的值
s.interpolate()

0      0.0
1      5.0
10    10.0
dtype: float64

In [29]:
# 和索引有关的线性插值，计算相应索引大小对应的值
s.interpolate(method='index')

0      0.0
1      1.0
10    10.0
dtype: float64

### 三、`Nullable`类型
#### 1. 缺失记号及其缺陷
在`numpy`中利用`np.nan`来表示缺失值，该元素除了不和其他任何元素相等之外，和自身的比较结果也返回`False`

In [30]:
np.nan == np.nan

False

In [31]:
np.nan == None

False

In [34]:
np.nan == False

False

虽然在对缺失序列或表格的元素进行比较操作的时候， `np.nan` 的对应位置会返回 `False` ，但是在使用 `equals` 函数进行两张表或两个序列的相同性检验时，会自动跳过两侧表都是缺失值的位置，直接返回 `True `

In [36]:
s1 = pd.Series([1, np.nan])
s2 = pd.Series([1, 2])
s3 = pd.Series([1, np.nan])

In [37]:
s1

0    1.0
1    NaN
dtype: float64

In [38]:
s2

0    1
1    2
dtype: int64

In [39]:
s3

0    1.0
1    NaN
dtype: float64

In [40]:
s1 == 1

0     True
1    False
dtype: bool

In [41]:
s1.equals(s2)

False

In [42]:
s1.equals(s3)

True

在时间序列的对象中，`pandas`利用`pd.NaT`来指代缺失值，作用和`np.nan`是一致的：

In [43]:
pd.to_timedelta(['30s', np.nan]) # Timedelta中的NaT

TimedeltaIndex(['0 days 00:00:30', NaT], dtype='timedelta64[ns]', freq=None)

In [44]:
pd.to_datetime(['20200101', np.nan]) # Datetime中的NaT

DatetimeIndex(['2020-01-01', 'NaT'], dtype='datetime64[ns]', freq=None)

> 那么为什么要引入 `pd.NaT` 来表示时间对象中的缺失呢？仍然以 `np.nan` 的形式存放会有什么问题？在 `pandas` 中可以看到 `object` 类型的对象，而 `object` 是一种混杂对象类型，如果出现了多个类型的元素同时存储在 `Series` 中，它的类型就会变成 `object` 。

In [45]:
pd.Series([1, 'two'])

0      1
1    two
dtype: object

`NaT`问题的根源来自于`np.nan`的本身是一种浮点类型，而如果浮点和时间类型混合存储，如果不设计新的内置缺失类型来处理，就会变成含糊不清的`object`类型。

In [46]:
type(np.nan)

float

由于`np.nan`的浮点性质，如果在一个整数的 `Series` 中出现缺失，那么其类型会转变为 `float64` ；而如果在一个布尔类型的序列中出现缺失，那么其类型就会转为 `object` 而不是 `bool`。 

In [49]:
pd.Series([1, np.nan]).dtype


dtype('float64')

In [48]:
pd.Series([True, False, np.nan]).dtype

dtype('O')

#### 2. `Nullable`类型的性质
从字面意义上看 Nullable 就是可空的，言下之意就是序列类型不受缺失值的影响。

In [50]:
pd.Series([np.nan, 1], dtype = 'Int64')

0    <NA>
1       1
dtype: Int64

In [51]:
pd.Series([np.nan, True], dtype = 'boolean')

0    <NA>
1    True
dtype: boolean

In [53]:
pd.Series([np.nan, 'my_str'], dtype = 'string')

0      <NA>
1    my_str
dtype: string

在`Int`的序列中，返回的结果会尽可能地成为`Nullable`的类型

In [54]:
pd.Series([np.nan, 0], dtype = 'Int64') + 1

0    <NA>
1       1
dtype: Int64

In [55]:
pd.Series([np.nan, 0], dtype = 'Int64') == 0

0    <NA>
1    True
dtype: boolean

In [56]:
pd.Series([np.nan, 0], dtype = 'Int64') * 0.5 # 只能是浮点

0    NaN
1    0.0
dtype: float64

对于`boolean`类型的序列而言，其和`bool`序列的行为主要有两点区别：
* 带有缺失的布尔列表无法进行索引器中的选择，而`boolean`会把缺失值看作`False`。
* 在进行逻辑运算时，`bool`类型在缺失处返回的永远是`False`，而`boolean`会根据逻辑运算是否能够确定唯一结果来返回相应的值。

>  `True | pd.NA` 中无论缺失值为什么值，必然返回 `True` ； `False | pd.NA` 中的结果会根据缺失值取值的不同而变化，此时返回 `pd.NA` ； `False & pd.NA` 中无论缺失值为什么值，必然返回 `False` 。


In [57]:
s = pd.Series(['a', 'b'])

s_bool = pd.Series([True, np.nan])

s_boolean = pd.Series([True, np.nan]).astype('boolean')


s[s_boolean]

0    a
dtype: object

In [58]:
s_boolean & True

0    True
1    <NA>
dtype: boolean

In [59]:
s_boolean | True

0    True
1    True
dtype: boolean

In [60]:
~s_boolean

0    False
1     <NA>
dtype: boolean

一般在实际数据处理时，可以在数据集读入后，先通过 convert_dtypes 转为 Nullable 类型:

In [61]:
df = pd.read_csv('data/learn_pandas.csv')

df = df.convert_dtypes()

df.dtypes

School          string
Grade           string
Name            string
Gender          string
Height         float64
Weight           Int64
Transfer        string
Test_Number      Int64
Test_Date       string
Time_Record     string
dtype: object

#### 3. 缺失数据的计算和分组
当调用函数`sum,prob`使用加法和乘法的时候，缺失数据等价于被分别视作0和1，级不改变原来的计算结果：

In [62]:
s = pd.Series([2,3,np.nan,4,5])
s.sum()


14.0

In [63]:
s.prod()

120.0

In [64]:
# 当使用累计函数时，会自动跳过缺失值所处的位置
s.cumsum()

0     2.0
1     5.0
2     NaN
3     9.0
4    14.0
dtype: float64

当进行单个标量运算的时候，除了 `np.nan ** 0` 和 `1 ** np.nan` 这两种情况为确定的值之外，所有运算结果全为缺失（ `pd.NA` 的行为与此一致 ），并且 `np.nan` 在比较操作时一定返回 `False` ，而 `pd.NA` 返回 `pd.NA `。

 `diff, pct_change` 这两个函数虽然功能相似，但是对于缺失的处理不同，前者凡是参与缺失计算的部分全部设为了缺失值，而后者缺失值位置会被设为 0% 的变化率

In [65]:
s.diff()

0    NaN
1    1.0
2    NaN
3    NaN
4    1.0
dtype: float64

In [66]:
s.pct_change()

0         NaN
1    0.500000
2    0.000000
3    0.333333
4    0.250000
dtype: float64

对于一些函数而言，缺失可以作为一个类别处理.

In [67]:
# 在groupby，get_dummies中可以设置相应的参数来进行增加缺失类别
df_nan = pd.DataFrame({'category':['a','a','b',np.nan,np.nan],
                      'value':[1,3,5,7,9]})
df_nan

,category,value
0,a,1
1,a,3
2,b,5
3,NaN,7
4,NaN,9


In [68]:
df_nan.groupby('category',
              dropna=False)['value'].mean()

category
a      2
b      5
NaN    8
Name: value, dtype: int64

In [69]:
pd.get_dummies(df_nan.category, dummy_na=True)

,a,b,NaN
0,1,0,0
1,1,0,0
2,0,1,0
3,0,0,1
4,0,0,1


#### Ex1：缺失值与类别的相关性检验
在数据处理中，含有过多缺失值的列往往会被删除，除非缺失情况与标签强相关。下面有一份关于二分类问题的数据集，其中 X_1, X_2 为特征变量， y 为二分类标签。

In [70]:
df = pd.read_csv('data/missing_chi.csv')

df.head()

,X_1,X_2,y
0,NaN,NaN,0
1,NaN,NaN,0
2,NaN,NaN,0
3,43.0,NaN,0
4,NaN,NaN,0


In [71]:
df.isna().mean()


X_1    0.855
X_2    0.894
y      0.000
dtype: float64

In [72]:
df.y.value_counts(normalize=True)

0    0.918
1    0.082
Name: y, dtype: float64

In [73]:
cat_1 = df.X_1.fillna('NaN').mask(df.X_1.notna()).fillna("NotNaN")

cat_2 = df.X_2.fillna('NaN').mask(df.X_2.notna()).fillna("NotNaN")

df_1 = pd.crosstab(cat_1, df.y, margins=True)

df_2 = pd.crosstab(cat_2, df.y, margins=True)

In [75]:
def compute_S(df):
  S = []
  for i in range(2):
      for j in range(2):
          E = df.iat[i, j]
          F = df.iat[i, 2]*df.iat[2, j]/df.iat[2,2]
          S.append((E-F)**2/F)
  return sum(S)

In [76]:
res1 = compute_S(df_1)

In [77]:
res2 = compute_S(df_2)

In [78]:
from scipy.stats import chi2

chi2.sf(res, 1)

array([       nan, 0.31731051, 0.31731051, 0.26355248,        nan,
       0.18587673, 0.15729921, 0.15729921,        nan])

In [79]:
chi2.sf(res2, 1)

7.459641265637543e-166